In [1]:
import polars as pl
import numpy as np

import os
import pandas as pd
os.getcwd()

'c:\\Users\\digom\\Documents\\GitHub\\fh-energia'

In [2]:
df_cnpj = pl.scan_csv('./data/estabelecimentos/estabelecimentos.csv', has_header=False, infer_schema_length=0)
if False:
    import polars as pl
    # Assuming 'df' is your DataFrame
    n = 1  # Change this to the number of rows you want to drop

    df_cnpj = df_cnpj.slice(n, len(df_cnpj) - n)


In [3]:
df_cnpj

In [4]:
situ_dict = {'01': 'Nula', '02': 'Ativa', '03': 'Suspensa', '04': 'Inapta', '08': 'Baixada'}

df_cnpj = df_cnpj.select('column_1', 'column_2', 'column_3', 'column_4', 'column_5', 'column_6',
                         'column_11',  'column_12', 'column_13',
                          'column_14', 'column_15', 'column_16', 'column_19', 'column_20', 'column_21')
df_cnpj = df_cnpj.with_columns(pl.concat_str(
        [
            pl.col("column_1"),
            pl.col("column_2"),
            pl.col("column_3"),
        ],
        separator="",
    ).alias('cnpj'),
    pl.concat_str(
        [
            pl.col("column_14"),
            pl.col("column_15"),
            pl.col("column_16"),
        ],
        separator=" ",
    ).alias('endereco'),
    pl.col('column_6').replace(situ_dict).alias('situacao_cadastral'))\
            .rename({'column_4': 'matriz_filial',  'column_19': 'CEP', 'column_12':'CNAE', 'column_13':'CNAE_2', 'column_16': 'numero', 'column_5': 'nome_fantasia',
                     'column_20': 'UF', 'column_21': 'municipio', 'column_11':'CNPJ_DAT_CON'})\
            .with_columns(cnae_2_split=pl.col("CNAE_2").str.split(","))\
            .with_columns(pl.col('CNPJ_DAT_CON').str.to_date('%Y%m%d').alias('CNPJ_DAT_CON'))\
            .with_columns(
                          pl.col('CNPJ_DAT_CON').dt.month().alias('CNPJ_DAT_CON_M'),
                          pl.col('CNPJ_DAT_CON').dt.year().alias('CNPJ_DAT_CON_Y'))\
            .explode('cnae_2_split')\
            .select(['cnpj', 'endereco', 'matriz_filial', 'CEP','CNAE', 'cnae_2_split', 'numero', 'situacao_cadastral', 
                     'nome_fantasia', 'UF', 'municipio', 'CNPJ_DAT_CON', 'CNPJ_DAT_CON_M', 'CNPJ_DAT_CON_Y'])\
            .filter(pl.col('situacao_cadastral').is_in(['Ativa', 'Suspensa', 'Inapta']))

In [5]:
#Filtrando pelo CEP de Sao Paulo
df_cnpj = df_cnpj.filter(pl.col("CEP").str.starts_with("1"))

In [6]:
df_cnpj.head(10).collect()

cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,municipio,CNPJ_DAT_CON,CNPJ_DAT_CON_M,CNPJ_DAT_CON_Y
str,str,str,str,str,str,str,str,str,str,str,date,i8,i32
"""76016369000316""","""RUA DO COMERCIO 55""","""2""","""11010141""","""4632001""",null,"""55""","""Inapta""",null,"""SP""","""7071""",1985-12-12,12,1985
"""52302726000182""","""RUA GREGORIO LUCHIARI 496""","""1""","""13472080""","""4712100""",null,"""496""","""Inapta""",null,"""SP""","""6131""",1983-02-23,2,1983
"""04785823000120""","""AVENIDA PADRE LEONARDO NUNES 1…","""1""","""11750000""","""1091102""","""4721103""","""175""","""Inapta""",null,"""SP""","""6853""",2001-11-14,11,2001
"""04785823000120""","""AVENIDA PADRE LEONARDO NUNES 1…","""1""","""11750000""","""1091102""","""5611204""","""175""","""Inapta""",null,"""SP""","""6853""",2001-11-14,11,2001
"""04785823000120""","""AVENIDA PADRE LEONARDO NUNES 1…","""1""","""11750000""","""1091102""","""5611203""","""175""","""Inapta""",null,"""SP""","""6853""",2001-11-14,11,2001
"""04377476000105""","""RUA ANOR RIBEIRO DE SOUZA 1115""","""1""","""14445000""","""1531901""","""1540800""","""1115""","""Inapta""",null,"""SP""","""6963""",2001-03-22,3,2001
"""58142886000198""","""RUA XAVIER PINHEIRO 253""","""1""","""11015090""","""4721102""","""4712100""","""253""","""Ativa""","""PANIFICADORA CAMOES""","""SP""","""7071""",1966-07-16,7,1966
"""07397393000168""","""AVENIDA PRESIDENTE KENNEDY 373…","""1""","""11703200""","""4321500""",null,"""3737""","""Inapta""",null,"""SP""","""6921""",2005-05-16,5,2005
"""07396902000138""","""RUA DOS SABIAS 23""","""1""","""13467304""","""4712100""",null,"""23""","""Inapta""","""MERCADO NOSSA SENHORA APARECID…","""SP""","""6131""",2005-05-09,5,2005


In [25]:
df_coord_sp = pl.scan_csv('./data/ibge/coord/35.csv', separator=";", infer_schema_length=0)
df_coord_sp = df_coord_sp.filter(~pl.col('COD_ESPECIE').is_in(["1", "2"]))


In [26]:
df_coord_sp.head(10).collect()

COD_UF,COD_MUN,COD_ESPECIE,LATITUDE,LONGITUDE,NV_GEO_COORD
str,str,str,str,str,str
"""35""","""3549904""","""6""","""-23.182593""","""-45.885680""","""1"""
"""35""","""3549904""","""6""","""-23.182709""","""-45.885500""","""1"""
"""35""","""3549904""","""7""","""-23.182776""","""-45.885490""","""1"""
"""35""","""3549904""","""6""","""-23.182753""","""-45.885488""","""1"""
"""35""","""3549904""","""6""","""-23.182720""","""-45.885609""","""1"""
"""35""","""3549904""","""6""","""-23.182735""","""-45.885435""","""1"""
"""35""","""3549904""","""6""","""-23.182765""","""-45.885333""","""1"""
"""35""","""3549904""","""6""","""-23.182753""","""-45.885260""","""1"""
"""35""","""3549904""","""6""","""-23.182735""","""-45.885230""","""1"""


In [9]:
df_bdgd = pl.scan_csv('data/bdgd/ucmt_pj.csv', separator=";", encoding='utf8-lossy', infer_schema_length=0)
df_bdgd = df_bdgd.with_columns(pl.col('CEP').str.replace('\.', '').str.replace('-', '').alias('CEP'),
                               pl.col('CNAE').str.replace('-', '').str.replace('/', '').alias('CNAE'),
                               pl.col('DAT_CON').str.to_date('%d/%m/%Y').alias('DAT_CON'),)\
                    .with_columns(
                               pl.col('DAT_CON').dt.month().alias('DAT_CON_M'),
                              pl.col('DAT_CON').dt.year().alias('DAT_CON_Y'))\
                    .filter((pl.col('LIV') == '0') & (pl.col("CEP").str.starts_with("1")))

In [10]:
import polars as pl
import numpy as np
from multiprocessing import Pool, cpu_count
from tqdm.notebook import tqdm

# Haversine distance function
import numpy as np

import numpy as np
import itertools




def haversine_np(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    
    All args must be of equal length.    
    
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6378.137 * c
    return km


def calculate_nearest(args):
    row1, df2, lat_col1, lon_col1, lat_col2, lon_col2 = args
    df2['index_2'] = np.arange(len(df2))
    #creating a triple with index_2, lat and lon
    df2 = df2.select([lat_col2, lon_col2, 'index_2'])
    lat1, lon1 = row1[lat_col1], row1[lon_col1]
    min_distance = float('inf')
    nearest_row = None
    #filtering df2 in order to get only the rows that have the interger rounded (2 decimals) lat and lon
    #df2 = df2[(df2[lat_col2].round(4) == lat1.round(4)) & (df2[lon_col2].round(4) == lon1.round(4))]
    """for row2 in df2:
        lat2, lon2 = row2[lat_col2], row2[lon_col2]
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance < min_distance:
            min_distance = distance
            nearest_row = row2"""
    dist_arr = list(itertools.product((lat1, lon1), df2))
    dist_arr = np.array(dist_arr)
    distances = haversine_np(dist_arr[:, 0], dist_arr[:, 1], dist_arr[:, 2], dist_arr[:, 3])
    min_distance = np.min(distances)
    nearest_row = df2[distances == min_distance]
    print("done")
    return (row1, nearest_row, min_distance)

# Function to join two LazyFrames with the nearest distance
def join_nearest_by_distance(lf1, lf2, lat_col1, lon_col1, lat_col2, lon_col2):
    # Collect the LazyFrames to DataFrames
    df1 = lf1.collect().to_numpy()
    df2 = lf2.collect().to_numpy()
    
    # Use multiprocessing to calculate distances and find the nearest
    with Pool(cpu_count()) as pool:
        distances = []
        for result in tqdm(pool.imap_unordered(calculate_nearest, [(row1, df2, lat_col1, lon_col1, lat_col2, lon_col2) for row1 in df1]), total=len(df1)):
            distances.append(result)
    
    # Create a new DataFrame with the results
    result = pl.DataFrame({
        **{f"{col}_1": [row1[i] for row1, _, _ in distances] for i, col in enumerate(lf1.columns)},
        **{f"{col}_2": [row2[i] for _, row2, _ in distances] for i, col in enumerate(lf2.columns)},
        "distance": [dist for _, _, dist in distances]
    })
    
    return result.lazy()


In [11]:
pd_bdgd = df_bdgd.collect().to_pandas()
cols_bdgd = pd_bdgd.columns

In [12]:
pd_bdgd.loc[pd_bdgd['CEP'] == '14790000', 'LGRD']

26726             CRT 041A SE ALIANCA 58
28532    AV JOSE QUINTINO DOS SANTOS 280
28535         CRT 022D SANTA HELENA S/N1
28565         CRT 022D SANTA HELENA S/N1
28615         CRT 040B ALBERTOPOLIS S/N1
                      ...               
34084         CRT 022C SANTA HELENA S/N1
34087         CRT 022D SANTA HELENA S/N1
34099         CRT 022C SANTA HELENA S/N1
34103            CRT 040A LGA FOGAO S/N1
34107         CRT 021D ALBERTOPOLIS S/N1
Name: LGRD, Length: 337, dtype: object

In [13]:
len(pd_bdgd)

41432

In [14]:
cod_id_set = []
one_cod_df_list = []
cnpj_cols = ['cnpj', 'nome_fantasia', 'situacao_cadastral', 'UF', 'matriz_filial']

# CEP e CNAE (filtrando o numero)

In [15]:
df_cnae_1 = df_cnpj.join(df_bdgd, on=['CEP', 'CNAE'], how='inner')
df_cnae_2 = df_cnpj.join(df_bdgd, left_on=['CEP', 'cnae_2_split'], right_on=['CEP', 'CNAE'], how='inner')
#concatenating the two dataframes
df_final = pl.concat([df_cnae_1, df_cnae_2])
df_filter = df_final.filter(pl.col('LGRD').str.contains(pl.col('numero'), literal=True))\
                    .unique(subset=['COD_ID', 'cnpj'])
df_cep_cnae_num = df_filter#.collect()#.to_pandas()
#getting top 1000 rows from df_cep_cnae_num
df_cep_cnae_num_top = df_cep_cnae_num.head(1000)


In [28]:
df_numpy_og = df_cep_cnae_num_top.select('POINT_Y', 'POINT_X').collect().to_numpy()
df_numpy_ibge = df_coord_sp.select('LATITUDE', 'LONGITUDE').collect().to_numpy()

In [31]:

df_coord_sp.select(pl.len()).collect()

len
u32
390927


In [29]:
from sklearn.metrics.pairwise import haversine_distances

distances = haversine_distances(df_numpy_og, df_numpy_ibge)

In [30]:
distances

array([[2.59851719, 2.59861256, 2.59867645, ..., 0.11118999, 0.10745791,
        0.1179745 ],
       [2.68751024, 2.68760217, 2.68766528, ..., 0.03981314, 0.03541364,
        0.0537886 ],
       [2.61632088, 2.61642385, 2.61648924, ..., 0.09339863, 0.09113343,
        0.09463299],
       ...,
       [2.78145992, 2.78157917, 2.78164627, ..., 0.12495197, 0.12952679,
        0.11267769],
       [1.12283211, 1.1227021 , 1.12265097, ..., 2.22277335, 2.22077269,
        2.23467035],
       [2.70476358, 2.70487139, 2.70493755, ..., 0.0421332 , 0.04547623,
        0.02849535]])

In [ ]:

result = join_nearest_by_distance(df_cep_cnae_num, df_coord_sp, "POINT_Y", "POINT_X", "LATITUDE", "LONGITUDE")
print(result.collect())

  0%|          | 0/390927 [00:00<?, ?it/s]

In [10]:
cep_cnae_num_cod_id =  (df_cep_cnae_num.COD_ID.value_counts() == 1).loc[lambda x: x == True].index.tolist()
cod_id_set = [*cod_id_set, *cep_cnae_num_cod_id]

In [11]:
one_cod_df_list.append(df_cep_cnae_num[df_cep_cnae_num.COD_ID.isin(cep_cnae_num_cod_id)][[*cnpj_cols, *cols_bdgd]])

In [12]:
len(set(cep_cnae_num_cod_id))

17359

# CEP e CNAE (sem filtrar numero)

In [13]:
df_cnae_1 = df_cnpj.join(df_bdgd, on=['CEP', 'CNAE'], how='inner')
df_cnae_2 = df_cnpj.join(df_bdgd, left_on=['CEP', 'cnae_2_split'], right_on=['CEP', 'CNAE'], how='inner')
#concatenating the two dataframes
df_final = pl.concat([df_cnae_1, df_cnae_2])
df_filter = df_final.unique(subset=['COD_ID', 'cnpj'])
df_cep_cnae = df_filter.collect().to_pandas()
df_cep_cnae

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,FIC_10,FIC_11,FIC_12,SEMRED,DESCR,DATA_BASE,POINT_X,POINT_Y,DAT_CON_M,DAT_CON_Y
0,07301012000103,RUA DOM HELIO CAMPOS 36,1,63800000,4754701,4755502,36,Ativa,MOBEL LAR,CE,...,0,0,0,0,J ALVES E OLIVEIRA LTDA,31DEC2022:00:00:00.0000000,-39.29397122,-5.19851026,3.0,2019.0
1,04215423000180,RUA FRANCISCO PINHEIRO 1-28,1,15520000,3101200,4930202,1-28,Ativa,CARMO-LAR,SP,...,0,0,0,0,None,31DEC2022:00:00:00.0000000,-50.07858777,-20.42206127,11.0,2001.0
2,04215423000180,RUA FRANCISCO PINHEIRO 1-28,1,15520000,3101200,4930202,1-28,Ativa,CARMO-LAR,SP,...,0,0,0,0,None,31DEC2022:00:00:00.0000000,-50.07816801,-20.42171012,6.0,1991.0
3,04215423000180,RUA FRANCISCO PINHEIRO 1-28,1,15520000,3101200,4930202,1-28,Ativa,CARMO-LAR,SP,...,0,0,0,0,None,31DEC2022:00:00:00.0000000,-50.07558149,-20.41701721,3.0,2007.0
4,04215423000180,RUA FRANCISCO PINHEIRO 1-28,1,15520000,3101200,4930202,1-28,Ativa,CARMO-LAR,SP,...,0,0,0,0,None,31DEC2022:00:00:00.0000000,-50.07989094,-20.4219867,10.0,1998.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3562695,63914865000183,RUA EUGENIO BUOSI 1 31,1,15520000,1622602,3101200,1 31,Ativa,BRILHANTE JM PORTAS,SP,...,0,0,0,0,None,31DEC2022:00:00:00.0000000,-50.07958096,-20.42082091,10.0,2010.0
3562696,63914865000183,RUA EUGENIO BUOSI 1 31,1,15520000,1622602,3101200,1 31,Ativa,BRILHANTE JM PORTAS,SP,...,0,0,0,0,None,31DEC2022:00:00:00.0000000,-50.07707999,-20.42149592,2.0,2012.0
3562697,63914865000183,RUA EUGENIO BUOSI 1 31,1,15520000,1622602,3101200,1 31,Ativa,BRILHANTE JM PORTAS,SP,...,0,0,0,0,None,31DEC2022:00:00:00.0000000,-50.07836067,-20.42175817,10.0,2000.0
3562698,63914865000183,RUA EUGENIO BUOSI 1 31,1,15520000,1622602,4754701,1 31,Ativa,BRILHANTE JM PORTAS,SP,...,0,0,0,0,None,31DEC2022:00:00:00.0000000,-50.07860972,-20.42381227,6.0,2004.0


In [14]:
cep_cnae_cod_id = (df_cep_cnae.COD_ID.value_counts() == 1).loc[lambda x: x == True].index.tolist()
cod_id_set = [*cod_id_set, *cep_cnae_cod_id]

In [15]:
one_cod_df_list.append(df_cep_cnae[df_cep_cnae.COD_ID.isin(cep_cnae_cod_id)][[*cnpj_cols, *cols_bdgd]])

# CEP, CNAE e DAT_CON

In [16]:
df_cnae_1 = df_cnpj.join(df_bdgd, right_on=['CEP', 'CNAE', 'DAT_CON'], left_on=['CEP', 'CNAE', 'CNPJ_DAT_CON'], how='inner')
df_cnae_2 = df_cnpj.join(df_bdgd, left_on=['CEP', 'cnae_2_split', 'CNPJ_DAT_CON'], right_on=['CEP', 'CNAE', 'DAT_CON'], how='inner')
#concatenating the two dataframes
df_final = pl.concat([df_cnae_1, df_cnae_2])
df_filter = df_final.unique(subset=['COD_ID', 'cnpj'])
df_cep_cnae_DAT = df_filter.collect().to_pandas()
df_cep_cnae_DAT

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,FIC_10,FIC_11,FIC_12,SEMRED,DESCR,DATA_BASE,POINT_X,POINT_Y,DAT_CON_M,DAT_CON_Y
0,00214242000340,RUA ERICH MEYER 108,2,89270000,1330800,None,108,Ativa,None,SC,...,0,0,0,0,"NR_CONTA_CNS_GENESIS: 42836184,9943223,7308699...",31DEC2022:00:00:00.0000000,-48.96267367,-26.47152776,12,2009
1,27784097000127,RUA OCEANICA AMABILE 736,1,11704420,4729699,None,736,Ativa,None,SP,...,1,0,0,1,None,31DEC2022:00:00:00.0000000,-46.48346521,-24.02339328,5,2017
2,42159396000164,RUA CAMPINAS 32,1,65930000,4781400,4761001,32,Inapta,LOJA EVANGELICA CORACAO DE DEUS,MA,...,0,0,1,0,0,31DEC2022:00:00:00.0000000,-47.50128856,-4.94902729,6,2021
3,15003231000126,FAZENDA SAGRADO CORACAO - ESTRADA DE GUAIRA AO...,1,14790000,0115600,0111302,S/N,Ativa,None,SP,...,1,3,3,1,None,31DEC2022:00:00:00.0000000,-48.42308987,-20.25902924,2,2012
4,35551964000102,TRAVESSA SANTO ANTONIO 78,1,45810000,5590699,9001902,78,Ativa,None,BA,...,0,0,0,0,1,31DEC2022:00:00:00.0000000,-39.06292521,-16.44604767,11,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,42505502000114,RUA 19 DE ABRIL 274,1,44695000,1359600,4713002,274,Ativa,PITICA DE LACO E AVIAMENTOS,BA,...,0,0,0,0,1,31DEC2022:00:00:00.0000000,-40.05725521,-11.36148589,6,2021
489,48569819000109,RODOVIA BR 242 KM 20 S/N,1,78888000,0115600,0111399,S/N,Ativa,AGROPECUARIA G5,MT,...,0,0,3,0,"2604956,",31DEC2022:00:00:00.0000000,-55.2779204,-13.25415069,11,2022
490,28164677000184,RUA HENRIQUE SAVI 55 QUADRA 15 15-55,1,17011900,4789001,4781400,15-55,Ativa,GABRIELA OLIVEIRA JOIAS E SEMI JOIAS,SP,...,0,0,0,1,None,31DEC2022:00:00:00.0000000,-49.04930596,-22.34173476,7,2017
491,39382611000187,10A AVENIDA MATO GROSSO S/N,1,78840000,4781400,4789001,S/N,Ativa,None,MT,...,1,0,0,1,None,31DEC2022:00:00:00.0000000,-55.18590454,-15.59443504,10,2020


In [17]:
cep_cnae_dat_cod_id = (df_cep_cnae_DAT.COD_ID.value_counts() == 1).loc[lambda x: x == True].index.tolist()
cod_id_set = [*cod_id_set, *cep_cnae_dat_cod_id]
df_cep_cnae_DAT.rename(columns={'CNPJ_DAT_CON': 'DAT_CON'}, inplace=True)
one_cod_df_list.append(df_cep_cnae_DAT[df_cep_cnae_DAT.COD_ID.isin(cep_cnae_dat_cod_id)][[*cnpj_cols, *cols_bdgd]])

# CEP, CNAE e DAT_COM (apenas mes e ano)

In [18]:
df_cnae_1 = df_cnpj.join(df_bdgd, left_on=['CEP', 'CNAE', 'CNPJ_DAT_CON_M', 'CNPJ_DAT_CON_Y'], right_on=['CEP', 'CNAE', 'DAT_CON_M', 'DAT_CON_Y'], how='inner')
df_cnae_2 = df_cnpj.join(df_bdgd, left_on=['CEP', 'cnae_2_split','CNPJ_DAT_CON_M', 'CNPJ_DAT_CON_Y'], right_on=['CEP', 'CNAE', 'DAT_CON_M', 'DAT_CON_Y'], how='inner')
#concatenating the two dataframes
df_final = pl.concat([df_cnae_1, df_cnae_2])
df_filter = df_final.unique(subset=['COD_ID', 'cnpj'])
df_cep_cnae_DAT_M = df_filter.collect().to_pandas()
df_cep_cnae_DAT_M

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,FIC_08,FIC_09,FIC_10,FIC_11,FIC_12,SEMRED,DESCR,DATA_BASE,POINT_X,POINT_Y
0,03606930000181,RUA DEZ DE NOVEMBRO 358,1,89825000,4930202,None,358,Ativa,MEMA TRANSPORTES,SC,...,1,0,1,0,0,0,None,31DEC2022:00:00:00.0000000,-52.56156644,-26.96487635
1,01993183000580,RUA BERNARDO SPECTOR 53,2,45810000,4711302,4691500,53,Ativa,ATACADO RONDELLI,BA,...,0,0,0,0,0,1,3,31DEC2022:00:00:00.0000000,-39.07750384,-16.44213064
2,59764886000192,AVENIDA BRASIL NORTE 1670,1,15385000,8411600,None,1670,Ativa,CAMARA MUNICIPAL DE ILHA SOLTEIRA,SP,...,0,0,0,0,1,0,None,31DEC2022:00:00:00.0000000,-51.33725709,-20.42862719
3,59764886000192,AVENIDA BRASIL NORTE 1670,1,15385000,8411600,None,1670,Ativa,CAMARA MUNICIPAL DE ILHA SOLTEIRA,SP,...,0,1,0,0,0,0,None,31DEC2022:00:00:00.0000000,-51.34139369,-20.42954207
4,08090198000153,SÍTIO SAO JOAO SN,1,15290000,0151202,0111399,SN,Ativa,None,SP,...,0,1,1,1,1,0,None,31DEC2022:00:00:00.0000000,-50.13529001,-21.08392949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,48560955000120,RUA SEBASTIAO AMADO 3078,1,68800000,4712100,4723700,3078,Ativa,None,PA,...,2,0,0,0,3,0,0,31DEC2022:00:00:00.0000000,-50.46712888,-1.66559167
12290,10734643002751,FAZENDA LAGOA DOS PATOS SN,2,14530000,0113000,0115600,SN,Ativa,None,SP,...,1,0,2,0,1,1,None,31DEC2022:00:00:00.0000000,-48.0820152,-20.20082742
12291,45078158000112,SITIO CURRAIS SN,1,62823000,0321302,0322102,SN,Ativa,AQUANORTE LABORATORIO DE POS LARVAS,CE,...,0,0,1,0,0,0,THIAGO FERREIRA LUSTOSA,31DEC2022:00:00:00.0000000,-37.78564681,-4.85658553
12292,03587113000123,RODOVIA RS 467 S/N,1,99950000,4661300,4683400,S/N,Ativa,ROTA AGRICOLA,RS,...,0,0,0,0,0,0,None,31DEC2022:00:00:00.0000000,-52.01954641,-28.07594696


In [19]:
cep_cnae_DAT_M_cod_id = (df_cep_cnae_DAT_M.COD_ID.value_counts() == 1).loc[lambda x: x == True].index.tolist()
cod_id_set = [*cod_id_set, *cep_cnae_DAT_M_cod_id]
df_cep_cnae_DAT_M.rename(columns={'CNPJ_DAT_CON_Y': 'DAT_CON_Y', 'CNPJ_DAT_CON_M': 'DAT_CON_M'}, inplace=True)
one_cod_df_list.append(df_cep_cnae_DAT_M[df_cep_cnae_DAT_M.COD_ID.isin(cep_cnae_DAT_M_cod_id)][[*cnpj_cols, *cols_bdgd]])

# CEP, CNAE e DAT_COM (apenas ano)

In [20]:
df_cnae_1 = df_cnpj.join(df_bdgd, left_on=['CEP', 'CNAE', 'CNPJ_DAT_CON_Y'], right_on=['CEP', 'CNAE', 'DAT_CON_Y'], how='inner')
df_cnae_2 = df_cnpj.join(df_bdgd, left_on=['CEP', 'cnae_2_split', 'CNPJ_DAT_CON_Y'], right_on=['CEP', 'CNAE', 'DAT_CON_Y'], how='inner')
#concatenating the two dataframes
df_final = pl.concat([df_cnae_1, df_cnae_2])
df_filter = df_final.unique(subset=['COD_ID', 'cnpj'])
df_cep_cnae_DAT_Y = df_filter.collect().to_pandas()
df_cep_cnae_DAT_Y

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,FIC_09,FIC_10,FIC_11,FIC_12,SEMRED,DESCR,DATA_BASE,POINT_X,POINT_Y,DAT_CON_M
0,03402398000180,RUA ADELAR MARIA DE ANDRADE 178,1,45810000,4711302,None,178,Inapta,None,BA,...,0,0,0,0,1,3,31DEC2022:00:00:00.0000000,-39.07750384,-16.44213064,12
1,01848456000179,RODOVIA C L 01 LIG. CAMPO LIMPO-MIRANAPOLIS S/N,1,75160000,2342702,None,S/N,Ativa,CERAMICA ELDORADO,GO,...,3,2,0,0,0,0,31DEC2022:00:00:00.0000000,-49.08378709,-16.3064507,1
2,05211637000140,RUA VALDEMIRO DA SILVA 236,1,88240000,1531901,1540800,236,Ativa,PITUXA,SC,...,0,0,0,0,0,"NR_CONTA_CNS_GENESIS: 26517060,10803961,730504...",31DEC2022:00:00:00.0000000,-48.85465982,-27.27062278,5
3,03585558000174,AVENIDA BENTO GONCALVES 1900,1,98465000,3211601,4789099,1900,Ativa,None,RS,...,0,1,0,1,0,None,31DEC2022:00:00:00.0000000,-53.18165093,-27.36433706,12
4,03587033000178,AVENIDA NEWTON PRADO 3476,1,13631040,4711302,None,3476,Ativa,SUPERMERCADO PEGUE MAIS,SP,...,1,2,0,0,0,None,31DEC2022:00:00:00.0000000,-47.42104605,-22.01080032,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147143,03810052000111,AVENIDA GETULIO VARGAS 2942,1,76868000,4721104,4649499,2942,Ativa,RONDOFEST COMERCIO VAREJISTA DE DOCES,RO,...,0,0,2,0,0,None,31DEC2022:00:00:00.0000000,-62.00698634,-9.4255588,1
147144,03603595002373,RUA GETULIO VARGAS 285,2,89600000,8800600,8591100,285,Ativa,CENTRO DE ATIVIDADES DE JOACABA,SC,...,0,0,0,0,0,"NR_CONTA_CNS_GENESIS: 29166676,10582166,730422...",31DEC2022:00:00:00.0000000,-51.52030161,-27.16887984,9
147145,04146233000158,AVENIDA MATO GROSSO 1502NE,1,78360000,9430800,9499500,1502NE,Ativa,ASSOC DE PAIS E PROF DA ESCOLA NOSSA SRA APARE...,MT,...,1,3,0,1,0,None,31DEC2022:00:00:00.0000000,-57.89568631,-13.69887398,5
147146,04744727000134,RODOVIA SC 410 6720,1,88240000,4782201,1531901,6720,Ativa,None,SC,...,0,1,1,1,0,"NR_CONTA_CNS_GENESIS: 25150163,10804300,731116...",31DEC2022:00:00:00.0000000,-48.86056418,-27.27707902,3


In [21]:
cep_cnae_DAT_Y_cod_id = (df_cep_cnae_DAT_Y.COD_ID.value_counts() == 1).loc[lambda x: x == True].index.tolist()
cod_id_set = [*cod_id_set, *cep_cnae_DAT_Y_cod_id]
df_cep_cnae_DAT_Y.rename(columns={'CNPJ_DAT_CON_Y': 'DAT_CON_Y'}, inplace=True)
one_cod_df_list.append(df_cep_cnae_DAT_Y[df_cep_cnae_DAT_Y.COD_ID.isin(cep_cnae_DAT_Y_cod_id)][[*cnpj_cols, *cols_bdgd]])

In [23]:
len(set(cod_id_set))

61098

In [26]:
new_col_id = cep_cnae_dat_cod_id
new_dat_m = [x for x in cep_cnae_DAT_M_cod_id if x not in new_col_id]
new_col_id = [*new_col_id, *new_dat_m]
new_dat_y = [x for x in cep_cnae_DAT_Y_cod_id if x not in new_col_id]
new_col_id = [*new_col_id, *new_dat_y]
new_cep_cnae_num = [x for x in cep_cnae_num_cod_id if x not in new_col_id]
new_col_id = [*new_col_id, *new_cep_cnae_num]
new_cep_cnae = [x for x in cep_cnae_cod_id if x not in new_col_id]
new_col_id = [*new_col_id, *new_cep_cnae]
len(new_col_id)


61098

In [ ]:
len(cep_cnae_cod_id)

In [50]:
df_concat = pd.concat([df_cep_cnae_DAT[df_cep_cnae_DAT.COD_ID.isin(cep_cnae_dat_cod_id)], 
                       df_cep_cnae_DAT_M[df_cep_cnae_DAT_M.COD_ID.isin(cep_cnae_DAT_M_cod_id)],
                       df_cep_cnae_DAT_Y[df_cep_cnae_DAT_Y.COD_ID.isin(cep_cnae_DAT_Y_cod_id)],
                       df_cep_cnae_num[df_cep_cnae_num.COD_ID.isin(cep_cnae_num_cod_id)], 
                       df_cep_cnae[df_cep_cnae.COD_ID.isin(cep_cnae_cod_id)],])
len(df_concat)

73188

In [61]:
df_concat.columns.tolist()

['cnpj',
 'endereco',
 'matriz_filial',
 'CEP',
 'CNAE',
 'cnae_2_split',
 'numero',
 'situacao_cadastral',
 'nome_fantasia',
 'UF',
 'municipio',
 'DAT_CON',
 'CNPJ_DAT_CON_M',
 'CNPJ_DAT_CON_Y',
 'COD_ID',
 'DIST',
 'PN_CON',
 'PAC',
 'CTMT',
 'UNI_TR_AT',
 'SUB',
 'CONJ',
 'MUN',
 'CEG_GD',
 'LGRD',
 'BRR',
 'CLAS_SUB',
 'TIP_CC',
 'FAS_CON',
 'GRU_TEN',
 'TEN_FORN',
 'GRU_TAR',
 'SIT_ATIV',
 'CAR_INST',
 'LIV',
 'ARE_LOC',
 'TIP_SIST',
 'DEM_CONT',
 'DEM_01',
 'DEM_02',
 'DEM_03',
 'DEM_04',
 'DEM_05',
 'DEM_06',
 'DEM_07',
 'DEM_08',
 'DEM_09',
 'DEM_10',
 'DEM_11',
 'DEM_12',
 'ENE_01',
 'ENE_02',
 'ENE_03',
 'ENE_04',
 'ENE_05',
 'ENE_06',
 'ENE_07',
 'ENE_08',
 'ENE_09',
 'ENE_10',
 'ENE_11',
 'ENE_12',
 'DIC_01',
 'DIC_02',
 'DIC_03',
 'DIC_04',
 'DIC_05',
 'DIC_06',
 'DIC_07',
 'DIC_08',
 'DIC_09',
 'DIC_10',
 'DIC_11',
 'DIC_12',
 'FIC_01',
 'FIC_02',
 'FIC_03',
 'FIC_04',
 'FIC_05',
 'FIC_06',
 'FIC_07',
 'FIC_08',
 'FIC_09',
 'FIC_10',
 'FIC_11',
 'FIC_12',
 'SEMRED',
 'DE

In [58]:
len(df_concat.drop_duplicates(subset=['cnpj', 'COD_ID']))

62894

In [54]:
len(df_concat.drop_duplicates(subset=['COD_ID']).drop_duplicates(subset=['cnpj']))

49023

In [57]:
len(df_concat.drop_duplicates(subset=['cnpj']).drop_duplicates(subset=['COD_ID']))

49146

In [66]:
df_cd_pj = df_concat.drop_duplicates(subset=['COD_ID']).drop_duplicates(subset=['cnpj'])
df_pj_cd = df_concat.drop_duplicates(subset=['cnpj']).drop_duplicates(subset=['COD_ID'])

#Keeping only the rows that are in both dataframes
df_final = pd.merge(df_cd_pj, df_pj_cd, on=df_concat.columns.tolist(), how='inner')
df_final.to_csv('data/energia_com_cnpj.csv', index=False)

In [55]:
len(df_concat.drop_duplicates(subset=['cnpj']))

50395

In [56]:
len(df_concat.drop_duplicates(subset=['COD_ID']))

61098

In [37]:
df_concat.to_csv('./data/energia_com_cnpj.csv', index=False)